In [ ]:
from nbdev import *

In [ ]:
%nbdev_default_export cli

Cells will be exported to drt.cli,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from drt.environment import DataIntakeEnv
from drt.create_environment import create_environment
from drt.registration import register_all
from drt.verification import check_datagroup
from drt.receipt import sync_data_folder
import drt.data_model as dm
from pathlib import Path
import argparse
import sys


def main(args=None):
    parser = argparse.ArgumentParser(description='Manage the data intake process.')
    subparsers = parser.add_subparsers(dest='mode')

    verify = subparsers.add_parser('verify', help='Verify an existing data intake folder')
    verify.add_argument('config_file', metavar='config_path', type=str, nargs='?',
                        default='.config/config.ini',
                        help='the path of the configuration file')
    verify.add_argument('-d', '--deep', action='store_true',
                        help='''calculate hashes when performing verification''')

    register = subparsers.add_parser('register', help='Register new deliveries and datasets')
    register.add_argument('config_file', metavar='config_path', type=str, nargs='?',
                          default='.config/config.ini',
                          help='the path of the configuration file')
    register.add_argument('-f', '--force', action='store_true',
                          help='''force the tool to re-calulate and re-register the system values''')

    create = subparsers.add_parser('create', help='Create a new data intake folder')
    create.add_argument('location', type=str, action='store', nargs='?', const='.',
                        help='create a new environment in the provided path')

    document = subparsers.add_parser('document',
                                     help='Build documentation based on information in db and provided by analysts')
    document.add_argument('config_file', metavar='config_path', type=str, nargs='?',
                          default='.config/config.ini',
                          help='the path of the configuration file')
    document.add_argument('-f', '--force', action='store_true',
                          help='force rebuild the documentation from the database')
    document.add_argument('-s', '--sync', action='store_true',
                          help='synchronise analyst changes to the database')

    if len(sys.argv) == 1:
        parser.print_help(sys.stderr)
        sys.exit(1)

    args = parser.parse_args()


    # Create new environment mode
    ##############################
    if args.mode == 'create':
        if not args.location or args.location == '.':
            config_file = create_environment(Path.cwd())
        elif args.location[0] == '/':
            config_file = create_environment(Path(args.location))
        else:
            config_file = create_environment((Path.cwd() / args.location))

        sys.exit()

    ##############################
    # Use environment modes
    ##############################
    config_file = args.config_file
    env = DataIntakeEnv(config_file)

    # Register mode
    ##############################
    if args.mode == 'register':
        if args.force:
            if input(
                    "!!! WARNING --force is a DESTRUCTIVE action, system information for specified dataset will be overwritten.\nIf no data groups are specified that means ALL data groups.\n\nAre you sure you want to continue? (Y/N)") == "Y":
                print("Activating forced recalculation")
                env.force_recalculate = True
            else:
                print("Proceeding without force, registering new data only.")

        # TODO add data group and data group type specific registration calls

        register_all(env)
        sys.exit()

    # Verify mode
    ##############################
    if args.mode == 'verify':
        if args.deep:
            [check_datagroup(env, group_type, light=False) for group_type in [dm.Delivery, dm.Raw_Data, dm.Dataset]]
        else:
            [check_datagroup(env, group_type) for group_type in [dm.Delivery, dm.Raw_Data, dm.Dataset]]
        sys.exit()

    # Generate docs mode
    ##############################
    if args.mode == 'document':
        if args.force:
            if input(
                    "!!! WARNING --force is a DESTRUCTIVE action, documentation will be overwritten.\nIf no data groups are specified that means ALL data groups.\n\nAre you sure you want to continue? (Y/N)") == "Y":
                print("Activating forced documentation rebuild")
                env.force_rebuild = True
            else:
                print("Proceeding without force, synchronising data and rebuilding.")
        if args.sync:
            [sync_data_folder(env, group_type) for group_type in [dm.Delivery, dm.Raw_Data, dm.Dataset]]
        sys.exit()

    # Clean up mode
    #############################
    if args.mode == 'clean':
        print("TBD: will allow you to purge the database if there are registered data groups that need to be removed.")

# if __name__ == "__main__":
#     sys.exit(main())